In [ ]:
import arviz as az
import pandas as pd
import bambi as bmb
import numpy as np 

: 

In [ ]:
data = pd.read_csv("./data/rt_tidy.csv")
data.head(10)

: 

In [ ]:
# create the new column that uses centralized rating
data['ratingcen'] = (data['rating'] - 5).abs() + 1
#transform country to region
region_mapping = {
    "KE": "Africa",
    "NG": "Africa",
    "ZA": "Africa",
    "CN": "East_Southeast_Asia",
    "IN": "East_Southeast_Asia",
    "MY": "East_Southeast_Asia",
    "TW": "East_Southeast_Asia",
    "TH": "East_Southeast_Asia",
    "AU": "Australia_New_Zealand",
    "NZ": "Australia_New_Zealand",
    "SV": "Central_America_Mexico",
    "MX": "Central_America_Mexico",
    "HU": "Eastern_Europe",
    "LT": "Eastern_Europe",
    "PL": "Eastern_Europe",
    "RU": "Eastern_Europe",
    "RS": "Eastern_Europe",
    "SK": "Eastern_Europe",
    "IR": "Middle_East",
    "IL": 	"Middle_East",
    'TR': 'Middle_East',
     'CA': 'United_States_Canada',
     'US': 'United_States_Canada',
     'DK': 'Scandinavia',
     'FI': 'Scandinavia',
     'NO': 'Scandinavia',
     'SE': 'Scandinavia',
     'AR': 'South_America',
     'BR': 'South_America',
     'CL': 'South_America',
     'CO': 'South_America',
     'EC': 'South_America',
     'GB-ENG': 'United_Kingdom',
     'GB-SCT': 'United_Kingdom',
     'GB-WLS': 'United_Kingdom',
     'AT': 	"Western_Europe",
     'BE': 	"Western_Europe",
     'FR': 	"Western_Europe",
     'DE': 	"Western_Europe",
     'GR': 	"Western_Europe",
     'IT': 	"Western_Europe",
     'NL': 	"Western_Europe",
     'PT': 	"Western_Europe",
     'ES': 	"Western_Europe",
     'CH': 	"Western_Europe"
}
data["region"] = data["country"].map(region_mapping)
#get rating equal 2 and 5
data2 = data[(data['ratingcen'] == 2) | (data['ratingcen'] == 5)]
#get trait unhappy
data2 = data2[data2['trait'] == 'unhappy']
#
data2['ratingcen'] = np.where(data2['ratingcen'] == 2, 0, 1)
data2.head(10)

: 

In [ ]:
model_non_hierarchical = bmb.Model('rating ~ rt + (rt | region)', data2, family="binomial")
model_non_hierarchical

: 

In [ ]:
idata_hierarchical = model_hierarchical.fit(random_seed=random_seed)

In [ ]:
az.plot_trace(idata_non_hierarchical, compact=False);

: 

In [ ]:
model_hierarchical = bmb.Model("p(H, AB) ~ 1 + (1|playerID)", df, family="binomial")
model_hierarchical

: 

In [ ]:
idata_hierarchical = model_hierarchical.fit(random_seed=random_seed)

In [ ]:
idata_prior = model_hierarchical.prior_predictive()
prior = az.extract_dataset(idata_prior, group="prior_predictive")["rating"]

In [ ]:
az.plot_forest(
    [idata_non_hierarchical, idata_hierarchical],
    var_names="rating",
    combined=True,
    colors=["#666666", RED],
    linewidth=2.6,
    markersize=8,
    ax=ax
)